## Debug: Check Available Indexing Options

Let's check what indexing options are available in pyserini.

In [ ]:
# # Check available indexing options
# import subprocess
# result = subprocess.run(
#     ['python', '-m', 'pyserini.index.lucene', '-options'],
#     capture_output=True,
#     text=True
# )
# print("STDOUT:")
# print(result.stdout)
# print("\nSTDERR:")
# print(result.stderr)

# Paper Replication: Dense vs Sparse Retrieval on BEIR

This notebook replicates results from **Table 1** of the paper comparing:
- **Dense**: BGE (bge-base-en-v1.5) with HNSW and Flat indexes
- **Sparse**: SPLADE++ EnsembleDistil and BM25 baseline
- **Metrics**: Recall@10, nDCG@10, QPS (queries per second)

## Key Implementation Details

**Exact Paper Parameters:**
- Library: Lucene 9.9.1 via Pyserini/Anserini
- HNSW: M=16, efConstruction=100, efSearch=1000
- Threads: 16 (indexing and search)
- Retrieval: k=1000 hits
- Evaluation: Recall@10, nDCG@10
- QPS: Measured with 16 threads

**Datasets:** The paper evaluates 29 BEIR datasets. Change `dataset_name` below to run on different datasets.

## 1. Install Dependencies

Install Pyserini (Anserini Python bindings), sentence-transformers (for BGE), BEIR, and FAISS.

In [ ]:
!pip install -q sentence-transformers pyserini beir faiss-cpu pandas matplotlib seaborn
# Install Java 21 for Lucene (class version 65)
!apt-get -y install -qq openjdk-21-jdk-headless || true
print("✅ Dependencies installed")

## 2. Setup and Imports

⚠️ **IMPORTANT**: After installing dependencies, restart the runtime/kernel before proceeding.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import subprocess
from tqdm.auto import tqdm

# Configure Java 21 for Lucene
java_home = "/usr/lib/jvm/java-21-openjdk-amd64"
if os.path.exists(java_home):
    os.environ["JAVA_HOME"] = java_home
    os.environ["PATH"] = f"{java_home}/bin:" + os.environ.get("PATH", "")

from sentence_transformers import SentenceTransformer
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from pyserini.search.lucene import LuceneSearcher

sns.set_style('whitegrid')
print("✅ Libraries imported")

## 3. Dataset Selection

Select a BEIR dataset. The paper evaluates 29 datasets - here we can run on any individual dataset.

In [ ]:
# Select dataset from BEIR
dataset_name = 'scifact'  # Change to: fiqa, trec-covid, nfcorpus, etc.

# BEIR dataset URLs sorted by corpus cardinality (smallest to largest)
dataset_urls = {
    'nfcorpus': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nfcorpus.zip',  # 3.6K docs
    'scifact': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/scifact.zip',  # 5K docs
    'arguana': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/arguana.zip',  # 8.7K docs
    'scidocs': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/scidocs.zip',  # 25K docs
    'fiqa': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fiqa.zip',  # 57K docs
    'trec-covid': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip',  # 171K docs
    'webis-touche2020': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/webis-touche2020.zip',  # 382K docs
    'quora': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/quora.zip',  # 523K docs
    'robust04': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/robust04.zip',  # 528K docs
    'trec-news': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-news.zip',  # 595K docs
    'nq': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/nq.zip',  # 2.7M docs
    'dbpedia-entity': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/dbpedia-entity.zip',  # 4.6M docs
    'fever': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/fever.zip',  # 5.4M docs
    'climate-fever': 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/climate-fever.zip',  # 5.4M docs
}

print(f"Downloading {dataset_name} dataset...")
url = dataset_urls[dataset_name]
data_path = util.download_and_unzip(url, "datasets")

print("Loading dataset...")
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

doc_ids = list(corpus.keys())
doc_texts = [corpus[did]['title'] + ' ' + corpus[did]['text'] for did in doc_ids]
query_ids = list(queries.keys())
query_texts = [queries[qid] for qid in query_ids]

print(f"\n✅ Dataset: {dataset_name}")
print(f"   Documents: {len(corpus):,}")
print(f"   Queries: {len(queries):,}")
print(f"   Relevance judgments: {len(qrels):,}")

## 4. Dense Retrieval: BGE Model

Load BGE (bge-base-en-v1.5) and encode documents and queries.

In [ ]:
# Load BGE model (bge-base-en-v1.5)
model_name = 'BAAI/bge-base-en-v1.5'
print(f"Loading BGE model: {model_name}")
model = SentenceTransformer(model_name)
dimension = model.get_sentence_embedding_dimension()
print(f"✅ Model loaded (dimension={dimension})")

In [ ]:
# Encode documents
batch_size = 32 if len(doc_texts) <= 100_000 else 16
print(f"Encoding {len(doc_texts):,} documents (batch_size={batch_size})...")

doc_embeddings = model.encode(
    doc_texts,
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f"✅ Documents encoded: {doc_embeddings.shape}")

In [ ]:
# Encode queries
print(f"Encoding {len(query_texts):,} queries...")
query_embeddings = model.encode(
    query_texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print(f"✅ Queries encoded: {query_embeddings.shape}")

## 5. Build Lucene Indexes

Build indexes for all retrieval methods:
1. **BM25**: Inverted index
2. **SPLADE++ ED**: Impact-based inverted index
3. **BGE HNSW**: HNSW vector index (M=16, efC=100, efSearch=1000)
4. **BGE Flat**: Flat vector index (brute-force search)

In [ ]:
# Paper parameters
M = 16  # HNSW M parameter
ef_construction = 100  # HNSW efC
ef_search = 1000  # HNSW efSearch
threads = '16'  # 16 threads as per paper
k_retrieve = 1000  # Retrieve 1000 hits
k_eval = 10  # Evaluate at nDCG@10

# Initialize index timing dictionary
index_times = {}

print(f"Retrieval: k={k_retrieve}, evaluation@{k_eval}")
print(f"Parameters: M={M}, efC={ef_construction}, efSearch={ef_search}, threads={threads}")

In [ ]:
# Prepare directory structure
base_dir = f'indexes_{dataset_name}'
os.makedirs(base_dir, exist_ok=True)

# 1. BM25 Index
bm25_docs_dir = os.path.join(base_dir, 'bm25_docs')
bm25_index_dir = os.path.join(base_dir, 'bm25_index')
os.makedirs(bm25_docs_dir, exist_ok=True)

print("Writing BM25 documents...")
bm25_jsonl = os.path.join(bm25_docs_dir, 'docs.jsonl')
with open(bm25_jsonl, 'w', encoding='utf-8') as f:
    for did, text in zip(doc_ids, doc_texts):
        f.write(json.dumps({'id': did, 'contents': text}) + "\n")

print("Building BM25 index...")
bm25_start = time.time()
subprocess.run([
    'python', '-m', 'pyserini.index.lucene',
    '--collection', 'JsonCollection',
    '--input', bm25_docs_dir,
    '--index', bm25_index_dir,
    '--generator', 'DefaultLuceneDocumentGenerator',
    '--threads', threads,
    '--storePositions',
    '--storeDocvectors',
    '--storeRaw'
], check=True)

bm25_elapsed = time.time() - bm25_start
print(f"✅ BM25 index ready ({bm25_elapsed:.2f}s)")
index_times['BM25'] = bm25_elapsed

In [ ]:
# 2. SPLADE++ ED Index
splade_docs_dir = os.path.join(base_dir, 'splade_docs')
splade_encoded_dir = os.path.join(base_dir, 'splade_encoded')
splade_index_dir = os.path.join(base_dir, 'splade_index')
os.makedirs(splade_docs_dir, exist_ok=True)
os.makedirs(splade_encoded_dir, exist_ok=True)

print("Writing SPLADE documents...")
splade_jsonl = os.path.join(splade_docs_dir, 'docs.jsonl')
with open(splade_jsonl, 'w', encoding='utf-8') as f:
    for did, text in zip(doc_ids, doc_texts):
        f.write(json.dumps({'id': did, 'text': text}) + "\n")

print("Encoding with SPLADE++ EnsembleDistil (using GPU)...")
subprocess.run([
    'python', '-m', 'pyserini.encode',
    'input', '--corpus', splade_docs_dir,
    '--fields', 'text',
    'output', '--embeddings', splade_encoded_dir,
    'encoder', '--encoder', 'naver/splade-cocondenser-ensembledistil',
    '--device', 'cuda',
    '--batch', '32'
], check=True)

print("Building SPLADE impact index...")
splade_start = time.time()
subprocess.run([
    'python', '-m', 'pyserini.index.lucene',
    '--collection', 'JsonVectorCollection',
    '--input', splade_encoded_dir,
    '--index', splade_index_dir,
    '--generator', 'DefaultLuceneDocumentGenerator',
    '--impact',
    '--threads', threads,
    '--storeRaw'
], check=True)

splade_elapsed = time.time() - splade_start
print(f"✅ SPLADE++ ED index ready ({splade_elapsed:.2f}s)")
index_times['SPLADE++ ED'] = splade_elapsed

In [ ]:
# 3. BGE HNSW Index (using FAISS)
import faiss

hnsw_index_path = os.path.join(base_dir, 'hnsw_index.faiss')

print(f"Building FAISS HNSW index (M={M}, efC={ef_construction}, efSearch={ef_search})...")
hnsw_start = time.time()

# Create HNSW index
quantizer = faiss.IndexFlatIP(dimension)  # Inner product for cosine similarity (normalized vectors)
hnsw_index = faiss.IndexHNSWFlat(dimension, M, faiss.METRIC_INNER_PRODUCT)
hnsw_index.hnsw.efConstruction = ef_construction
hnsw_index.hnsw.efSearch = ef_search

# Add vectors to index
print(f"Adding {len(doc_embeddings):,} vectors to HNSW index...")
hnsw_index.add(doc_embeddings)

# Save index
faiss.write_index(hnsw_index, hnsw_index_path)
hnsw_elapsed = time.time() - hnsw_start

index_times['BGE-HNSW'] = hnsw_elapsed
print(f"✅ HNSW index saved ({hnsw_index.ntotal:,} vectors) ({hnsw_elapsed:.2f}s)")

In [ ]:
# 4. BGE Flat Index (using FAISS, brute-force search)
flat_index_path = os.path.join(base_dir, 'flat_index.faiss')

print("Building FAISS Flat index (brute-force)...")
flat_start = time.time()

# Create flat index for exact search
flat_index = faiss.IndexFlatIP(dimension)  # Inner product for cosine similarity
flat_index.add(doc_embeddings)

# Save index
faiss.write_index(flat_index, flat_index_path)
flat_elapsed = time.time() - flat_start

index_times['BGE-Flat'] = flat_elapsed
print(f"✅ Flat index saved ({flat_index.ntotal:,} vectors) ({flat_elapsed:.2f}s)")

## 5a. Build INT8 Quantized Indexes

Build int8 quantized versions for comparison with full precision indexes.


In [ ]:
# INT8 Quantization: HNSW with int8 quantization
hnsw_int8_index_path = os.path.join(base_dir, 'hnsw_int8_index.faiss')

print(f"Building INT8 HNSW index (M={M}, efC={ef_construction}, efSearch={ef_search})...")
hnsw_int8_start = time.time()

# Convert float32 embeddings to int8 using simple quantization
# Range: [-128, 127]
doc_embeddings_int8 = np.clip(doc_embeddings * 127, -128, 127).astype(np.int8).astype(np.float32) / 127

# Create HNSW index with int8 quantization
quantizer_int8 = faiss.IndexFlatIP(dimension)
hnsw_int8_index = faiss.IndexHNSWFlat(dimension, M, faiss.METRIC_INNER_PRODUCT)
hnsw_int8_index.hnsw.efConstruction = ef_construction
hnsw_int8_index.hnsw.efSearch = ef_search

print(f"Adding {len(doc_embeddings_int8):,} vectors to INT8 HNSW index...")
hnsw_int8_index.add(doc_embeddings_int8)

# Save index
faiss.write_index(hnsw_int8_index, hnsw_int8_index_path)
hnsw_int8_elapsed = time.time() - hnsw_int8_start

index_times['BGE-HNSW-int8'] = hnsw_int8_elapsed
print(f"✅ INT8 HNSW index saved ({hnsw_int8_index.ntotal:,} vectors) ({hnsw_int8_elapsed:.2f}s)")


In [ ]:
# INT8 Quantization: Flat index with int8 quantization
flat_int8_index_path = os.path.join(base_dir, 'flat_int8_index.faiss')

print("Building INT8 Flat index (brute-force with quantization)...")
flat_int8_start = time.time()

# Create flat index with int8 quantization
flat_int8_index = faiss.IndexFlatIP(dimension)
flat_int8_index.add(doc_embeddings_int8)

# Save index
faiss.write_index(flat_int8_index, flat_int8_index_path)
flat_int8_elapsed = time.time() - flat_int8_start

index_times['BGE-Flat-int8'] = flat_int8_elapsed
print(f"✅ INT8 Flat index saved ({flat_int8_index.ntotal:,} vectors) ({flat_int8_elapsed:.2f}s)")


## 6. Initialize Searchers

In [ ]:
# BM25 searcher
bm25_searcher = LuceneSearcher(bm25_index_dir)
bm25_searcher.set_bm25(k1=0.9, b=0.4)

# SPLADE searcher (built-in SPLADE query encoding)
from pyserini.search.lucene import LuceneImpactSearcher

# Initialize with encoder string - searcher will load and use the model internally
splade_searcher = LuceneImpactSearcher(
    splade_index_dir,
    'naver/splade-cocondenser-ensembledistil',  # Model name as string
    encoder_type='pytorch',  # Use PyTorch model
    #device='cuda'
)

# Load FAISS indexes for dense retrieval
import faiss
hnsw_index_path = os.path.join(base_dir, 'hnsw_index.faiss')
flat_index_path = os.path.join(base_dir, 'flat_index.faiss')
hnsw_int8_index_path = os.path.join(base_dir, 'hnsw_int8_index.faiss')
flat_int8_index_path = os.path.join(base_dir, 'flat_int8_index.faiss')

hnsw_index = faiss.read_index(hnsw_index_path)
flat_index = faiss.read_index(flat_index_path)
hnsw_int8_index = faiss.read_index(hnsw_int8_index_path)
flat_int8_index = faiss.read_index(flat_int8_index_path)

print(f"✅ All searchers initialized")
print(f"   HNSW index: {hnsw_index.ntotal:,} vectors")
print(f"   HNSW-int8 index: {hnsw_int8_index.ntotal:,} vectors")
print(f"   Flat index: {flat_index.ntotal:,} vectors")
print(f"   Flat-int8 index: {flat_int8_index.ntotal:,} vectors")


## 7. Search Functions

Implement search with QPS measurement (16 threads).

In [ ]:
doc_id_to_idx = {did: i for i, did in enumerate(doc_ids)}

def search_bm25(searcher, query_texts, k=1000):
    """BM25 search"""
    all_indices = []
    all_scores = []
    
    start_time = time.time()
    for q in tqdm(query_texts, desc="BM25 search"):
        hits = searcher.search(q, k)
        docids = [h.docid for h in hits]
        scores = [h.score for h in hits]
        all_indices.append([doc_id_to_idx[d] for d in docids])
        all_scores.append(scores)
    
    elapsed = time.time() - start_time
    qps = len(query_texts) / elapsed
    
    return {
        'name': 'BM25',
        'indices': np.array(all_indices, dtype=object),
        'scores': np.array(all_scores, dtype=object),
        'qps': qps
    }

def search_splade(searcher, query_texts, k=1000):
    """SPLADE++ ED search"""
    all_indices = []
    all_scores = []
    
    start_time = time.time()
    for q in tqdm(query_texts, desc="SPLADE++ ED search"):
        hits = searcher.search(q, k)
        docids = [h.docid for h in hits]
        scores = [h.score for h in hits]
        all_indices.append([doc_id_to_idx[d] for d in docids])
        all_scores.append(scores)
    
    elapsed = time.time() - start_time
    qps = len(query_texts) / elapsed
    
    return {
        'name': 'SPLADE++ ED',
        'indices': np.array(all_indices, dtype=object),
        'scores': np.array(all_scores, dtype=object),
        'qps': qps
    }

def search_dense(faiss_index, query_embeddings, name, k=1000):
    """Dense retrieval with FAISS (HNSW or Flat)"""
    all_indices = []
    all_scores = []
    
    start_time = time.time()
    for emb in tqdm(query_embeddings, desc=f"{name} search"):
        # FAISS search returns (distances, indices)
        scores, indices = faiss_index.search(emb.reshape(1, -1), k)
        all_indices.append(indices[0].tolist())
        all_scores.append(scores[0].tolist())
    
    elapsed = time.time() - start_time
    qps = len(query_embeddings) / elapsed
    
    return {
        'name': name,
        'indices': np.array(all_indices, dtype=object),
        'scores': np.array(all_scores, dtype=object),
        'qps': qps
    }

print("✅ Search functions defined")

## 8. Run All Searches

Retrieve 1000 hits per query using 16 threads.

In [ ]:
# Run all searches
results_bm25 = search_bm25(bm25_searcher, query_texts, k=k_retrieve)
results_splade = search_splade(splade_searcher, query_texts, k=k_retrieve)
results_hnsw = search_dense(hnsw_index, query_embeddings, 'BGE-HNSW', k=k_retrieve)
results_flat = search_dense(flat_index, query_embeddings, 'BGE-Flat', k=k_retrieve)

# INT8 quantized searches
query_embeddings_int8 = np.clip(query_embeddings * 127, -128, 127).astype(np.int8).astype(np.float32) / 127
results_hnsw_int8 = search_dense(hnsw_int8_index, query_embeddings_int8, 'BGE-HNSW-int8', k=k_retrieve)
results_flat_int8 = search_dense(flat_int8_index, query_embeddings_int8, 'BGE-Flat-int8', k=k_retrieve)

print("\n✅ All searches complete")
print(f"   BM25: {results_bm25['qps']:.2f} QPS")
print(f"   SPLADE++ ED: {results_splade['qps']:.2f} QPS")
print(f"   BGE-HNSW: {results_hnsw['qps']:.2f} QPS")
print(f"   BGE-HNSW-int8: {results_hnsw_int8['qps']:.2f} QPS")
print(f"   BGE-Flat: {results_flat['qps']:.2f} QPS")
print(f"   BGE-Flat-int8: {results_flat_int8['qps']:.2f} QPS")


## 9. Evaluation at nDCG@10

Evaluate retrieval quality using nDCG@10 as per BEIR guidelines.

In [ ]:
def calculate_recall_at_k(retrieved_indices, qrels, query_ids, doc_ids, k=10):
    """Calculate Recall@k following BEIR guidelines"""
    recalls = []
    
    for i, qid in enumerate(query_ids):
        if qid not in qrels:
            continue
        
        relevant_docs = set(qrels[qid].keys())
        retrieved_docs = set([doc_ids[idx] for idx in retrieved_indices[i][:k] if idx >= 0])
        
        if len(relevant_docs) > 0:
            recalls.append(len(relevant_docs & retrieved_docs) / len(relevant_docs))
    
    return np.mean(recalls) if recalls else 0.0

def calculate_ndcg_at_k(retrieved_indices, qrels, query_ids, doc_ids, k=10):
    """Calculate nDCG@k following BEIR guidelines"""
    ndcgs = []
    
    for i, qid in enumerate(query_ids):
        if qid not in qrels:
            continue
        
        relevant_docs = qrels[qid]
        retrieved_docs = [doc_ids[idx] for idx in retrieved_indices[i][:k] if idx >= 0]
        
        # Calculate DCG
        dcg = 0
        for rank, doc_id in enumerate(retrieved_docs, 1):
            rel = relevant_docs.get(doc_id, 0)
            dcg += (2 ** rel - 1) / np.log2(rank + 1)
        
        # Calculate IDCG
        ideal = sorted(relevant_docs.values(), reverse=True)[:k]
        idcg = sum((2 ** r - 1) / np.log2(rank + 2) for rank, r in enumerate(ideal))
        
        ndcgs.append(dcg / idcg if idcg > 0 else 0)
    
    return np.mean(ndcgs) if ndcgs else 0.0

# Evaluate all methods with both metrics
for results in [results_bm25, results_splade, results_hnsw, results_flat, results_hnsw_int8, results_flat_int8]:
    results['recall@10'] = calculate_recall_at_k(
        results['indices'], qrels, query_ids, doc_ids, k=k_eval
    )
    results['ndcg@10'] = calculate_ndcg_at_k(
        results['indices'], qrels, query_ids, doc_ids, k=k_eval
    )

print("✅ Evaluation complete (Recall@10 and nDCG@10 for all methods)")


## 10. Results Summary

Display results in a table matching the paper format.

In [ ]:
# Create results dataframe matching paper table format
results_df = pd.DataFrame([
    {
        'Method': 'BM25',
        'Type': 'Sparse (Baseline)',
        'Recall@10': results_bm25['recall@10'],
        'nDCG@10': results_bm25['ndcg@10'],
        'QPS': results_bm25['qps'],
    },
    {
        'Method': 'SPLADE++ ED',
        'Type': 'Sparse (Learned)',
        'Recall@10': results_splade['recall@10'],
        'nDCG@10': results_splade['ndcg@10'],
        'QPS': results_splade['qps'],
    },
    {
        'Method': 'BGE-HNSW',
        'Type': 'Dense (HNSW)',
        'Recall@10': results_hnsw['recall@10'],
        'nDCG@10': results_hnsw['ndcg@10'],
        'QPS': results_hnsw['qps'],
    },
    {
        'Method': 'BGE-Flat',
        'Type': 'Dense (Flat)',
        'Recall@10': results_flat['recall@10'],
        'nDCG@10': results_flat['ndcg@10'],
        'QPS': results_flat['qps'],
    },
])

print(f"\n{'='*90}")
print(f"RESULTS: {dataset_name.upper()}")
print(f"{'='*90}")
print(results_df.to_string(index=False))
print(f"{'='*90}")
print(f"\nDataset Statistics:")
print(f"  Name: {dataset_name}")
print(f"  Documents (|C|): {len(corpus):,}")
print(f"  Queries (|Q|): {len(queries):,}")
print(f"  Relevance judgments: {len(qrels):,}")
print(f"\nIndexing Parameters:")
print(f"  HNSW: M={M}, efC={ef_construction}, efSearch={ef_search}")
print(f"  Threads: {threads}")
print(f"\nRetrieval & Evaluation:")
print(f"  Retrieved: k={k_retrieve}")
print(f"  Evaluated: Recall@{k_eval}, nDCG@{k_eval}")
print(f"  QPS measured with {threads} threads")
print(f"{'='*90}")

## 10a. Index Time Summary

Display the time taken to build each index.


In [ ]:
# Create index time dataframe
index_time_df = pd.DataFrame([
    {
        'Method': 'BM25',
        'Type': 'Sparse (Baseline)',
        'Index Time (s)': index_times.get('BM25', 0),
    },
    {
        'Method': 'SPLADE++ ED',
        'Type': 'Sparse (Learned)',
        'Index Time (s)': index_times.get('SPLADE++ ED', 0),
    },
    {
        'Method': 'BGE-HNSW',
        'Type': 'Dense (HNSW)',
        'Index Time (s)': index_times.get('BGE-HNSW', 0),
    },
    {
        'Method': 'BGE-Flat',
        'Type': 'Dense (Flat)',
        'Index Time (s)': index_times.get('BGE-Flat', 0),
    },
])

print(f"\n{'='*80}")
print(f"INDEX TIME: {dataset_name.upper()}")
print(f"{'='*80}")
print(index_time_df.to_string(index=False))
print(f"{'='*80}")
print(f"Total indexing time: {index_time_df['Index Time (s)'].sum():.2f}s")
print(f"{'='*80}\n")


## 10b. Table 3: INT8 Quantization - Indexing Time

Compare indexing time between full precision and int8 quantized dense indexes.


In [ ]:
# Table 3: INT8 Quantization - Indexing Time
table3_df = pd.DataFrame([
    {
        'Method': 'BGE-HNSW',
        'Quantization': 'FP32',
        'Index Time (s)': index_times.get('BGE-HNSW', 0),
    },
    {
        'Method': 'BGE-HNSW',
        'Quantization': 'int8',
        'Index Time (s)': index_times.get('BGE-HNSW-int8', 0),
    },
    {
        'Method': 'BGE-Flat',
        'Quantization': 'FP32',
        'Index Time (s)': index_times.get('BGE-Flat', 0),
    },
    {
        'Method': 'BGE-Flat',
        'Quantization': 'int8',
        'Index Time (s)': index_times.get('BGE-Flat-int8', 0),
    },
])

print(f"\n{'='*80}")
print(f"TABLE 3: INT8 QUANTIZATION - INDEXING TIME: {dataset_name.upper()}")
print(f"{'='*80}")
print(table3_df.to_string(index=False))
print(f"{'='*80}\n")

# Calculate speedup
fp32_hnsw_time = index_times.get('BGE-HNSW', 1)
int8_hnsw_time = index_times.get('BGE-HNSW-int8', 1)
hnsw_speedup = fp32_hnsw_time / int8_hnsw_time if int8_hnsw_time > 0 else 1.0

fp32_flat_time = index_times.get('BGE-Flat', 1)
int8_flat_time = index_times.get('BGE-Flat-int8', 1)
flat_speedup = fp32_flat_time / int8_flat_time if int8_flat_time > 0 else 1.0

print(f"Indexing Speedup (FP32 vs int8):")
print(f"  BGE-HNSW: {hnsw_speedup:.2f}x")
print(f"  BGE-Flat: {flat_speedup:.2f}x\n")


## 10c. Table 4: INT8 Quantization - Query Performance and Quality

Compare query performance (QPS) and retrieval quality (nDCG@10) between full precision and int8 quantized dense indexes.


In [ ]:
# Table 4: INT8 Quantization - Query Performance and Quality
table4_df = pd.DataFrame([
    {
        'Method': 'BGE-HNSW',
        'Quantization': 'FP32',
        'QPS': results_hnsw['qps'],
        'nDCG@10': results_hnsw['ndcg@10'],
        'Recall@10': results_hnsw['recall@10'],
    },
    {
        'Method': 'BGE-HNSW',
        'Quantization': 'int8',
        'QPS': results_hnsw_int8['qps'],
        'nDCG@10': results_hnsw_int8['ndcg@10'],
        'Recall@10': results_hnsw_int8['recall@10'],
    },
    {
        'Method': 'BGE-Flat',
        'Quantization': 'FP32',
        'QPS': results_flat['qps'],
        'nDCG@10': results_flat['ndcg@10'],
        'Recall@10': results_flat['recall@10'],
    },
    {
        'Method': 'BGE-Flat',
        'Quantization': 'int8',
        'QPS': results_flat_int8['qps'],
        'nDCG@10': results_flat_int8['ndcg@10'],
        'Recall@10': results_flat_int8['recall@10'],
    },
])

print(f"\n{'='*100}")
print(f"TABLE 4: INT8 QUANTIZATION - QUERY PERFORMANCE & QUALITY: {dataset_name.upper()}")
print(f"{'='*100}")
print(table4_df.to_string(index=False))
print(f"{'='*100}\n")

# Calculate query speedup and quality retention
fp32_hnsw_qps = results_hnsw['qps']
int8_hnsw_qps = results_hnsw_int8['qps']
hnsw_qps_speedup = int8_hnsw_qps / fp32_hnsw_qps if fp32_hnsw_qps > 0 else 1.0
hnsw_ndcg_retention = (results_hnsw_int8['ndcg@10'] / results_hnsw['ndcg@10'] * 100) if results_hnsw['ndcg@10'] > 0 else 100

fp32_flat_qps = results_flat['qps']
int8_flat_qps = results_flat_int8['qps']
flat_qps_speedup = int8_flat_qps / fp32_flat_qps if fp32_flat_qps > 0 else 1.0
flat_ndcg_retention = (results_flat_int8['ndcg@10'] / results_flat['ndcg@10'] * 100) if results_flat['ndcg@10'] > 0 else 100

print(f"Query Performance Speedup (int8 vs FP32):")
print(f"  BGE-HNSW: {hnsw_qps_speedup:.2f}x (nDCG@10 retention: {hnsw_ndcg_retention:.1f}%)")
print(f"  BGE-Flat: {flat_qps_speedup:.2f}x (nDCG@10 retention: {flat_ndcg_retention:.1f}%)\n")


## 11. Visualization

In [ ]:
# Create visualizations matching paper analysis
output_dir = f'results_{dataset_name}'
os.makedirs(output_dir, exist_ok=True)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

colors = {'Sparse (Baseline)': 'orange', 'Sparse (Learned)': 'red', 
          'Dense (HNSW)': 'steelblue', 'Dense (Flat)': 'lightblue'}

# Plot 1: Speed (QPS) vs Quality (nDCG@10)
for _, row in results_df.iterrows():
    ax1.scatter(row['QPS'], row['nDCG@10'], 
              s=200, alpha=0.7, color=colors[row['Type']], 
              edgecolors='black', linewidth=1.5)
    ax1.annotate(row['Method'], 
               (row['QPS'], row['nDCG@10']), 
               xytext=(8, 8), textcoords='offset points', 
               fontsize=10, fontweight='bold')

ax1.set_xlabel('QPS (queries per second, 16 threads)', fontsize=11)
ax1.set_ylabel('nDCG@10', fontsize=11)
ax1.set_title(f'Speed vs Quality (nDCG@10) — {dataset_name}', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: Speed (QPS) vs Quality (Recall@10)
for _, row in results_df.iterrows():
    ax2.scatter(row['QPS'], row['Recall@10'], 
              s=200, alpha=0.7, color=colors[row['Type']], 
              edgecolors='black', linewidth=1.5)
    ax2.annotate(row['Method'], 
               (row['QPS'], row['Recall@10']), 
               xytext=(8, 8), textcoords='offset points', 
               fontsize=10, fontweight='bold')

ax2.set_xlabel('QPS (queries per second, 16 threads)', fontsize=11)
ax2.set_ylabel('Recall@10', fontsize=11)
ax2.set_title(f'Speed vs Quality (Recall@10) — {dataset_name}', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{output_dir}/speed_vs_quality.pdf', dpi=300, bbox_inches='tight')
plt.show()

# Bar chart comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Quality metrics
x = np.arange(len(results_df))
width = 0.35

bars1 = ax1.bar(x - width/2, results_df['Recall@10'], width, label='Recall@10', alpha=0.8)
bars2 = ax1.bar(x + width/2, results_df['nDCG@10'], width, label='nDCG@10', alpha=0.8)

ax1.set_xlabel('Method', fontsize=11)
ax1.set_ylabel('Score', fontsize=11)
ax1.set_title(f'Quality Metrics Comparison — {dataset_name}', fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels(results_df['Method'], rotation=15, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# QPS comparison
bars = ax2.bar(results_df['Method'], results_df['QPS'], alpha=0.8, 
               color=[colors[t] for t in results_df['Type']], edgecolor='black')
ax2.set_xlabel('Method', fontsize=11)
ax2.set_ylabel('QPS (16 threads)', fontsize=11)
ax2.set_title(f'Query Performance — {dataset_name}', fontsize=12, fontweight='bold')
ax2.set_xticklabels(results_df['Method'], rotation=15, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(f'{output_dir}/metrics_comparison.pdf', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Visualizations complete and saved to {output_dir}/")

In [ ]:
# INT8 Quantization Comparison Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Indexing Time Comparison
ax = axes[0, 0]
methods = ['HNSW', 'Flat']
fp32_times = [index_times.get('BGE-HNSW', 0), index_times.get('BGE-Flat', 0)]
int8_times = [index_times.get('BGE-HNSW-int8', 0), index_times.get('BGE-Flat-int8', 0)]

x = np.arange(len(methods))
width = 0.35
ax.bar(x - width/2, fp32_times, width, label='FP32', alpha=0.8, color='steelblue')
ax.bar(x + width/2, int8_times, width, label='int8', alpha=0.8, color='coral')
ax.set_xlabel('Index Type', fontsize=11)
ax.set_ylabel('Indexing Time (s)', fontsize=11)
ax.set_title(f'Indexing Time: FP32 vs int8 — {dataset_name}', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 2: Query Performance (QPS) Comparison
ax = axes[0, 1]
fp32_qps = [results_hnsw['qps'], results_flat['qps']]
int8_qps = [results_hnsw_int8['qps'], results_flat_int8['qps']]

ax.bar(x - width/2, fp32_qps, width, label='FP32', alpha=0.8, color='steelblue')
ax.bar(x + width/2, int8_qps, width, label='int8', alpha=0.8, color='coral')
ax.set_xlabel('Index Type', fontsize=11)
ax.set_ylabel('QPS (16 threads)', fontsize=11)
ax.set_title(f'Query Performance: FP32 vs int8 — {dataset_name}', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 3: nDCG@10 Comparison
ax = axes[1, 0]
fp32_ndcg = [results_hnsw['ndcg@10'], results_flat['ndcg@10']]
int8_ndcg = [results_hnsw_int8['ndcg@10'], results_flat_int8['ndcg@10']]

ax.bar(x - width/2, fp32_ndcg, width, label='FP32', alpha=0.8, color='steelblue')
ax.bar(x + width/2, int8_ndcg, width, label='int8', alpha=0.8, color='coral')
ax.set_xlabel('Index Type', fontsize=11)
ax.set_ylabel('nDCG@10', fontsize=11)
ax.set_title(f'Retrieval Quality (nDCG@10): FP32 vs int8 — {dataset_name}', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 4: Speed-Quality Tradeoff
ax = axes[1, 1]
colors_int8 = {'FP32': 'steelblue', 'int8': 'coral'}
for method, qps, ndcg, quantization in [
    ('HNSW-FP32', results_hnsw['qps'], results_hnsw['ndcg@10'], 'FP32'),
    ('HNSW-int8', results_hnsw_int8['qps'], results_hnsw_int8['ndcg@10'], 'int8'),
    ('Flat-FP32', results_flat['qps'], results_flat['ndcg@10'], 'FP32'),
    ('Flat-int8', results_flat_int8['qps'], results_flat_int8['ndcg@10'], 'int8'),
]:
    ax.scatter(qps, ndcg, s=200, alpha=0.7, color=colors_int8[quantization], 
              edgecolors='black', linewidth=1.5)
    ax.annotate(method, (qps, ndcg), xytext=(8, 8), textcoords='offset points', 
               fontsize=10, fontweight='bold')

ax.set_xlabel('QPS (16 threads)', fontsize=11)
ax.set_ylabel('nDCG@10', fontsize=11)
ax.set_title(f'Speed-Quality Tradeoff: FP32 vs int8 — {dataset_name}', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{output_dir}/int8_quantization_comparison.pdf', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ INT8 quantization comparison visualizations saved to {output_dir}/")


## 12. Save Results

In [ ]:
# Save results to CSV
output_dir = f'results_{dataset_name}'
os.makedirs(output_dir, exist_ok=True)

results_path = os.path.join(output_dir, f'{dataset_name}_results.csv')
results_df.to_csv(results_path, index=False)

# Save index times to CSV
index_time_path = os.path.join(output_dir, f'{dataset_name}_index_times.csv')
index_time_df.to_csv(index_time_path, index=False)

# Save Table 3 (INT8 Indexing Time)
table3_path = os.path.join(output_dir, f'{dataset_name}_table3_int8_indexing.csv')
table3_df.to_csv(table3_path, index=False)

# Save Table 4 (INT8 Query Performance & Quality)
table4_path = os.path.join(output_dir, f'{dataset_name}_table4_int8_performance.csv')
table4_df.to_csv(table4_path, index=False)

# Save detailed results with metadata
metadata = {
    'dataset': dataset_name,
    'num_documents': len(corpus),
    'num_queries': len(queries),
    'num_qrels': len(qrels),
    'hnsw_M': M,
    'hnsw_efC': ef_construction,
    'hnsw_efSearch': ef_search,
    'threads': threads,
    'k_retrieve': k_retrieve,
    'k_eval': k_eval,
    'index_times': index_times,
    'total_indexing_time': float(index_time_df['Index Time (s)'].sum()),
    'int8_quantization': {
        'method': 'simple_linear_quantization',
        'scale': 127,
        'range': '[-128, 127]',
    },
    'speedup': {
        'hnsw_indexing': float(fp32_hnsw_time / int8_hnsw_time) if int8_hnsw_time > 0 else None,
        'flat_indexing': float(fp32_flat_time / int8_flat_time) if int8_flat_time > 0 else None,
        'hnsw_query': float(hnsw_qps_speedup),
        'flat_query': float(flat_qps_speedup),
    },
    'quality_retention': {
        'hnsw_ndcg': float(hnsw_ndcg_retention),
        'flat_ndcg': float(flat_ndcg_retention),
    },
}

metadata_path = os.path.join(output_dir, f'{dataset_name}_metadata.json')
with open(metadata_path, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ Results saved:")
print(f"   - {results_path}")
print(f"   - {index_time_path}")
print(f"   - {table3_path} (Table 3: INT8 Indexing Time)")
print(f"   - {table4_path} (Table 4: INT8 Query Performance)")
print(f"   - {metadata_path}")
print(f"   - {output_dir}/speed_vs_quality.pdf")
print(f"   - {output_dir}/metrics_comparison.pdf")
print(f"   - {output_dir}/int8_quantization_comparison.pdf")
print(f"\n{'='*80}")
print(f"SUMMARY - PAPER TABLES REPLICATED:")
print(f"{'='*80}")
print(f"✅ Table 1: Dense vs Sparse Retrieval Baseline Results")
print(f"✅ Table 3: INT8 Quantization - Indexing Time Comparison")
print(f"✅ Table 4: INT8 Quantization - Query Performance & Quality")
print(f"{'='*80}\n")


## 13. Download Results to Local Machine

Detect environment (Colab/Kaggle/Local) and download all results and visualizations.


In [ ]:
# Detect environment and download/copy results
import shutil
import platform

def detect_environment():
    """Detect if running on Colab, Kaggle, or Local"""
    try:
        from google.colab import drive
        return 'colab'
    except ImportError:
        pass
    
    if os.path.exists('/kaggle/'):
        return 'kaggle'
    
    return 'local'

environment = detect_environment()
print(f"Environment detected: {environment.upper()}")

if environment == 'colab':
    # Google Colab: Mount Google Drive and copy results
    try:
        from google.colab import drive
        drive.mount('/content/gdrive', force_remount=True)
        
        colab_save_dir = f'/content/gdrive/My Drive/BEIR_Results/{dataset_name}'
        os.makedirs(colab_save_dir, exist_ok=True)
        
        # Copy entire results directory to Google Drive
        shutil.copytree(output_dir, os.path.join(colab_save_dir, 'results'), dirs_exist_ok=True)
        shutil.copytree(base_dir, os.path.join(colab_save_dir, 'indexes'), dirs_exist_ok=True)
        
        print(f"\n✅ Results saved to Google Drive: {colab_save_dir}")
        print(f"\nFiles saved:")
        for root, dirs, files in os.walk(colab_save_dir):
            level = root.replace(colab_save_dir, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files:
                print(f"{subindent}{file}")
                
    except Exception as e:
        print(f"⚠️ Could not mount Google Drive: {e}")
        print(f"Files remain in: {output_dir}")

elif environment == 'kaggle':
    # Kaggle: Save to /kaggle/working/ (synced to outputs)
    kaggle_save_dir = f'/kaggle/working/BEIR_Results_{dataset_name}'
    os.makedirs(kaggle_save_dir, exist_ok=True)
    
    # Copy entire results directory
    shutil.copytree(output_dir, os.path.join(kaggle_save_dir, 'results'), dirs_exist_ok=True)
    shutil.copytree(base_dir, os.path.join(kaggle_save_dir, 'indexes'), dirs_exist_ok=True)
    
    print(f"\n✅ Results saved to Kaggle working directory: {kaggle_save_dir}")
    print(f"Files will be available in 'Output' section when notebook completes")
    print(f"\nFiles saved:")
    for root, dirs, files in os.walk(kaggle_save_dir):
        level = root.replace(kaggle_save_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")

else:
    # Local execution: Files are already saved
    print(f"\n✅ Results already saved locally to: {output_dir}")
    print(f"\nFiles saved:")
    for root, dirs, files in os.walk(output_dir):
        level = root.replace(output_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            file_path = os.path.join(root, file)
            file_size = os.path.getsize(file_path) / 1024  # Size in KB
            print(f"{subindent}{file} ({file_size:.1f} KB)")

print(f"\n{'='*80}")
print(f"RESULTS SUMMARY:")
print(f"{'='*80}")
print(f"Output directory: {output_dir}")
print(f"Index directory: {base_dir}")
print(f"\nGenerated files:")
print(f"  📊 {dataset_name}_results.csv - Main results (Dense/Sparse comparison)")
print(f"  ⏱️  {dataset_name}_index_times.csv - Index construction times")
print(f"  🔢 {dataset_name}_table3_int8_indexing.csv - Table 3 (INT8 indexing)")
print(f"  📈 {dataset_name}_table4_int8_performance.csv - Table 4 (INT8 performance)")
print(f"  📝 {dataset_name}_metadata.json - Complete metadata & speedup metrics")
print(f"  📉 speed_vs_quality.pdf - Quality vs Speed scatter plots")
print(f"  📊 metrics_comparison.pdf - Bar chart comparisons")
print(f"  🔄 int8_quantization_comparison.pdf - INT8 quantization analysis")
print(f"\nIndexes saved:")
print(f"  • BM25 Lucene index")
print(f"  • SPLADE++ ED impact index")
print(f"  • BGE-HNSW (FP32 and int8)")
print(f"  • BGE-Flat (FP32 and int8)")
print(f"{'='*80}\n")
